In [1]:
import math
import os
import xml.etree.ElementTree as ET
from glob import glob

import cv2
import numpy as np
from keras.applications import InceptionResNetV2
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.utils import Sequence
from keras.optimizers import Adam


In [2]:
images_path = '../../dataset/images/*.png'
annotations_path = '../../dataset/annotations/*.xml'


In [3]:
def get_annotations(annotation, image):
    xml = ET.parse(annotation)
    info = xml.getroot().find('object').find('bndbox')
    xmin = int(info.find('xmin').text)
    ymin = int(info.find('ymin').text)
    xmax = int(info.find('xmax').text)
    ymax = int(info.find('ymax').text)

    xmin = xmin / image.shape[1]
    xmax = xmax / image.shape[1]
    ymin = ymin / image.shape[0]
    ymax = ymax / image.shape[0]

    return np.array([xmin, ymin, xmax, ymax])


In [4]:
class Dataset(Sequence):
    def __init__(self, images_path: str, annotations_path: str, start: float, end: float, batch_size: int = 32):
        self.images_path = glob(images_path)
        self.images_path = self.images_path[int(start * len(self.images_path)):int(end * len(self.images_path))]
        self.annotations_path = glob(annotations_path)
        self.annotations_path = self.annotations_path[int(start * len(self.annotations_path)):int(end * len(self.annotations_path))]
        self.start = start
        self.end = end
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.images_path) / self.batch_size)

    def __getitem__(self, index):
        images = self.images_path[index * self.batch_size:(index + 1) * self.batch_size]
        annotations = self.annotations_path[index * self.batch_size:(index + 1) * self.batch_size]

        images = [cv2.imread(os.path.join('../../dataset/images', i)) for i in images]
        annotations = [get_annotations(a, i) for a, i in zip(annotations, images)]
        images = [cv2.resize(i, (224, 224)) / 255 for i in images]

        return np.array(images), np.array(annotations)


In [5]:
model = Sequential()
model.add(InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(4, activation='sigmoid'))

model.summary()


KeyboardInterrupt: 

In [ ]:
batch_size = 64
train_dataset = Dataset(images_path, annotations_path, 0, 0.8, batch_size)
val_dataset = Dataset(images_path, annotations_path, 0.8, 0.9, batch_size)

model.compile(optimizer=Adam(1e-4), loss='mse')
history = model.fit(x=train_dataset, epochs=15, batch_size=batch_size, validation_data=val_dataset)


Epoch 1/15
6/6 [==============================] - 23s 1s/step - loss: 0.0888 - val_loss: 0.0752
Epoch 2/15
6/6 [==============================] - 4s 654ms/step - loss: 0.0390 - val_loss: 0.0394
Epoch 3/15
6/6 [==============================] - 4s 690ms/step - loss: 0.0164 - val_loss: 0.0285
Epoch 4/15
6/6 [==============================] - 4s 636ms/step - loss: 0.0081 - val_loss: 0.0232
Epoch 5/15
6/6 [==============================] - 4s 627ms/step - loss: 0.0048 - val_loss: 0.0219
Epoch 6/15
6/6 [==============================] - 4s 662ms/step - loss: 0.0031 - val_loss: 0.0228
Epoch 7/15
6/6 [==============================] - 4s 630ms/step - loss: 0.0026 - val_loss: 0.0199
Epoch 8/15
6/6 [==============================] - 4s 675ms/step - loss: 0.0014 - val_loss: 0.0212
Epoch 9/15
6/6 [==============================] - 5s 794ms/step - loss: 0.0011 - val_loss: 0.0200
Epoch 10/15
6/6 [==============================] - 5s 703ms/step - loss: 9.7459e-04 - val_loss: 0.0196
Epoch 11/15
6/6 [

In [ ]:
model.save('./model_15.h5')
